In [5]:
import numpy as np
import json
import re
import pandas

with open("Biobank_Bounding_Boxes_2021_9_6_mobilenet_v3.json","r") as f:
    networkOutput = json.load(f)

with open("UK Biobank large test n19695.json","r") as f:
    targetData = json.load(f)

In [10]:
targets = [float(x["t1_slice_location"]) for x in targetData]
eids = [str(x["id"]) for x in targetData]
targetDict = dict(zip(eids,targets))

gtEids = list(networkOutput.keys())
trainEids = pandas.read_csv("train_new_format.csv",header=None)
valEids = pandas.read_csv("val_new_format.csv",header=None)

trainEids = list(trainEids[0])
valEids = list(valEids[0])

trainEids.extend(valEids)

gtEids = [x for x in gtEids if x not in trainEids]
print("Number of GT Eids = {}".format(len(gtEids)))

Number of GT Eids = 64


In [14]:
def volume(inp):
    vol = np.zeros((inp.shape[0],inp.shape[1]//4))
    for i,arr in enumerate(inp):
        for j in range(0,inp.shape[1],4):
            vol[i,j//4] = (arr[j+2]-arr[j])*(arr[j+3]-arr[j+1])
    return vol

In [18]:
keysOI = ["Liver","Body","Lung","Heart","Kidney","Spleen","IVC","Aorta"]
bbInput = np.zeros((len(gtEids),(len(keysOI)+2)*4))

for i,eid in enumerate(gtEids):
    bBoxes = networkOutput[eid]
    bBKeys = list(bBoxes.keys())
    j = 0
    for kOI in keysOI:
        patt = str("{}\_\d+".format(kOI))
        numOrgs = []
        
        for bb in bBKeys:
            org = re.findall(patt,bb)
            numOrgs.extend(org)

        maxPred = 0.0
        predAcc = np.array([x[4] for x in numOrgs])
        numOrgs = np.array(numOrgs)

        if kOI in ["Lung","Kidney"]:
            topPred = predAcc.argsort()[-2:]
            tempKeys = numOrgs[topPred]
            tempBbox = np.array([bBoxes[tK][:4] for tK in tempKeys])
            tempBbox = tempBbox.flatten()
            bbInput[i,j:j+(4*len(tempKeys))] = tempBbox

            j += 8
        else:
            try:
                topPred = predAcc.argsort()[-1:]
                tempKey = numOrgs[topPred.astype(int)][0]
                tempBbox = bBoxes[tempKey][:4]
                bbInput[i,j:j+4] = tempBbox
            except IndexError:
                pass

            j += 4

In [20]:
vol = volume(bbInput)

In [22]:
def vol_2_dict(vol,eids):
    newVol = {}
    keysOI = ["Liver","Body","Lung","Heart","Kidney","Spleen","IVC","Aorta"]
    for eid,arr in zip(eids,vol):
        tempDict = {}

        i = 0
        j = 0
        while i < arr.shape[0]:

            if i == 2 or i == 5:
                val = np.mean(arr[i:i+1])
                if val < 0.01:
                    val = np.nan
                tempDict[keysOI[j]] = val
                i += 2
            else:
                val = arr[i]
                if val < 0.01:
                    val = np.nan
                tempDict[keysOI[j]] = val
                i += 1

            j += 1

        newVol[eid] = tempDict

    return newVol     

volDict = vol_2_dict(vol,gtEids)

In [23]:
trainRatioDict = {}
for eid in volDict.keys():
    tempDict = {}
    for org in keysOI:
        tempDict[org] = volDict[eid][org]/volDict[eid]["Body"]
    trainRatioDict[eid] = tempDict

ratioDf = pandas.DataFrame(trainRatioDict)
ratioDf = ratioDf.T
ratioDf

,Liver,Body,Lung,Heart,Kidney,Spleen,IVC,Aorta
1652105_20204_2_0,0.214002,1.0,0.129862,0.048188,0.047530,0.009918,0.009019,0.007763
3941648_20204_2_0,0.130906,1.0,0.147933,0.149891,NaN,NaN,NaN,0.007158
1014070_20204_2_0,0.306001,1.0,NaN,NaN,NaN,0.030551,0.002988,0.007981
1014104_20204_2_0,0.316020,1.0,NaN,NaN,NaN,0.087448,0.003758,0.010406
1014221_20204_2_0,0.292820,1.0,0.084993,0.063648,0.006424,0.051534,NaN,0.008862
...,...,...,...,...,...,...,...,...
3442768_20204_2_0,0.120822,1.0,0.248956,0.124147,NaN,0.119545,NaN,0.007698
3635639_20204_2_0,NaN,1.0,0.081229,NaN,NaN,NaN,NaN,NaN
3826689_20204_2_0,0.082791,1.0,NaN,NaN,0.038755,NaN,NaN,0.007618
3840226_20204_2_0,NaN,1.0,0.133308,0.144726,NaN,NaN,NaN,0.007859
